In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker as tk
import scipy as sp

rPath = "RESULTS/variations/"

In [ ]:
# It is important that these be arrays because they need to be
# in the same order as the parameters.h file
pops = ["V1L4E", "V1L4I", "V1L23E", "V1L23I", "V2L4E", "V2L4I", "V2L23E", "V2L23I", "LGN"]
dims = {"V1L4E":[23,4], "V1L4I":[23,1], "V1L23E":[17,7], "V1L23I":[17,2], "V2L4E":[13,10], "V2L4I":[13,3], "V2L23E":[10,16], "V2L23I":[10,5], "LGN":[32,2]}
RFsv = {"FF":{"V1L4":10, "V1L23":7, "V2L4":5, "V2L23":4}, "Lat":{"V1L4":11, "V1L23":9, "V2L4":7, "V2L23":5}, "FB":{"V1L4":7, "V1L23":8, "V2L4":4, "V2L23":0}}
syns = [ ["LGN", "V1L4E", "FF"], ["LGN", "V1L4I", "FF"], ["V1L4E", "V1L23E", "FF"], ["V1L4E", "V1L23I", "FF"], 
    ["V1L23E", "V2L4E", "FF"], ["V1L23E", "V2L4I", "FF"], ["V2L4E", "V2L23E", "FF"], ["V2L4E", "V2L23I", "FF"],
    ["V1L4E", "V1L4I", "Lat"], ["V1L4I", "V1L4E", "Lat"], ["V1L4I", "V1L4I", "Lat"], ["V1L23E", "V1L23I", "Lat"],
    ["V1L23I", "V1L23E", "Lat"], ["V1L23I", "V1L23I", "Lat"], ["V2L4E", "V2L4I", "Lat"], ["V2L4I", "V2L4E", "Lat"], 
    ["V2L4I", "V2L4I", "Lat"], ["V2L23E", "V2L23I", "Lat"], ["V2L23I", "V2L23E", "Lat"], ["V2L23I", "V2L23I", "Lat"], 
    ["V2L23E", "V1L23E", "FB"], ["V2L23E", "V1L23I", "FB"],  ["V1L23E", "V1L4I", "FB"], ["V2L23E", "V2L4I", "FB"], 
    ["V1L4I", "V1L23E", "FF"], ["V1L4I", "V1L23I", "FF"], ["V2L4I", "V2L23E", "FF"], ["V2L4I", "V2L23I", "FF"] ]
sNames = [ s[0]+"-"+s[1] for s in syns]

vars = ["trace", "average"]

#  Training Results

In [ ]:
rAvg = {}
wAvg = {}
rHist = {}
for i, v in enumerate(vars):
    rAvg[v] = pd.read_csv(rPath+v+"/training/avgRates.tsv", sep="\t")
    wAvg[v] = pd.read_csv(rPath+v+"/training/avgWeights.tsv", sep="\t")
    # Load distirbutions
    dist = {}
    for p in pops:
        dist[p] = {"count":[], "bins":[]}
        with open(rPath+v+"/training/"+p+"dist.vec") as f:
            for i, r in enumerate(f):
                if (i%2 == 0): dist[p]["count"].append(np.fromstring(r, sep=" "))
                else: dist[p]["bins"].append(np.fromstring(r, sep=" "))
        rHist[v] = dist
            

In [ ]:
tA = {v:{} for v in vars}
tTH = {v:{} for v in vars}
for v in vars:
    for p in pops[:-1]:
        tA[v][p] = np.loadtxt(rPath+v+"/training/a"+p)
        tTH[v][p] = np.loadtxt(rPath+v+"/training/theta"+p)

## Network Activity

In [ ]:
# Activity levels
fig, axs = plt.subplots(2,2, figsize = (13,7))
for i,v in enumerate(vars):
    # Smoothed rates
    rAvg[v][pops].apply(lambda x : sp.signal.savgol_filter(x,20,2)).plot(ax=axs[i][0], legend=False)
    axs[i][0].set_title(v+" target")
    # Signal retention
    rAvg[v][pops[:-1]].apply(lambda x : x/rAvg[v][pops[-1]]).apply(lambda x : sp.signal.savgol_filter(x,20,2)).plot(ax=axs[i][1])
    axs[i][1].set_title("input normalised signal")
    axs[i][1].legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
fig.tight_layout()
fig.show()


#plt.savefig(rPath+"training_sum/avgRates.png")

In [ ]:
# No Activity
fig, axs = plt.subplots(3,1, figsize = (6,4))
for i,v in enumerate(vars):
    # No activity
    rAvg[v].where(rAvg[v]==0)[pops[:-1]].apply(lambda x : x+.1*(pops.index(x.name))).plot(ax=axs[i], legend=False)
    axs[i].set_xlim(0,200)
    axs[i].set_yticks([])
    axs[i].set_ylabel(v)

axs[0].set_xticks([])
axs[1].set_xticks([])
h,l = axs[0].get_legend_handles_labels()
fig.legend(h,l, loc='center left', bbox_to_anchor=(.95, .67), )
fig.suptitle("Zero-average activity")
fig.tight_layout()
fig.show()



In [ ]:
# Average last 10 records
#pd.concat([rAvg[v][pops[:-1]].tail(10).mean() for v in vars],axis=1, keys=vars)
pd.concat([rAvg[v][pops[:-1]].tail(10).agg(["mean", "std"]).transpose() for v in vars],axis=1, keys=vars)

In [ ]:
# Activity Distributions
for v in vars:
    fig, axs = plt.subplots(len(pops[:-1]),4, figsize=(20,10))
    toshow = [1, 50, 100, 200]
    for k,p in enumerate(pops[:-1]):
        h = rHist[v][p]
        axs[k][0].text(-.2,.5,p, transform = axs[k][0].transAxes)
        for i, t in enumerate(toshow):
            axs[k][i].bar(h["bins"][t][1:-1], h["count"][t][1:], width=h["bins"][t][1]*.8, align='center')
            axs[k][i].bar(rAvg[v][p].values[t], max(h["count"][t][1:]), width=h["bins"][t][1]*.6, color="red")
            perc = 100*(1.-h["count"][t][0]/sum(h["count"][t]))
            axs[k][i].text(.9,.8, str(round(perc,2))+"%" if perc > .01  else "> .01%", fontsize=8,transform=axs[k][i].transAxes)

    for i, t in enumerate(toshow): axs[0][i].set_title(str(t*1000) + " patches")
    fig.suptitle(v)
    fig.tight_layout()
    fig.show()

## Plastic Parameters

In [ ]:
# Intrinsic plasticity
for v in vars:
    fig, axs = plt.subplots(4,2, figsize = (10,5))
    for i,p in enumerate(pops[:4]):
        c,b,_ = axs[i][0].hist(tA[v][p], 40)
        axs[i][0].bar(tA[v][p].mean(), c.max(),width=(b[1]-b[0])*.3, color="red")
        axs[i][0].text(-.25,.5,p, transform = axs[i][0].transAxes)
        c,b,_ = axs[i][1].hist(tTH[v][p], 40)
        axs[i][1].bar(tTH[v][p].mean(), c.max(),width=(b[1]-b[0])*.3, color="red")
    axs[0][0].set_title("a")
    axs[0][1].set_title(r'$\theta$')
    fig.suptitle(v)
    fig.tight_layout()
    fig.show()


In [ ]:
ipStats = []
for v in vars:
    meanA = []
    meanT = []
    stdA = []
    stdT = []
    for p in pops[:-1]:
        meanA.append(tA[v][p].mean())
        stdA.append(tA[v][p].std())
        meanT.append(tTH[v][p].mean())
        stdT.append(tTH[v][p].std())
    A = pd.DataFrame({"mean":meanA, "std":stdA}, index=pops[:-1])
    T = pd.DataFrame({"mean":meanT, "std":stdT}, index=pops[:-1])
    ipStats.append(pd.concat([A,T], axis=1, keys=["a","theta"]))
ipStats = pd.concat(ipStats, axis=1, keys=vars)
ipStats

In [ ]:
# Average weights
for v in vars:
    fig,axs = plt.subplots(int(len(pops)/3), 3, figsize = (12, 6))
    for i,p in enumerate(pops):
        cols = [c for c in wAvg[v].columns if p+"-" in c]
        wAvg[v][cols].plot(ax=axs[int(i/3)][i%3])
        axs[int(i/3)][i%3].set_title(p, fontsize=10)
        axs[int(i/3)][i%3].legend(title="Target", labels=[c.split("-")[1] for c in cols], fontsize=8)
    fig.suptitle(v, fontsize=11)
    fig.tight_layout()
    fig.show()

In [ ]:
# Weights distributions
for v in vars[:1]:
    fig,axs = plt.subplots(int(len(syns)/4), 4, figsize = (15, 20))
    for i,s in enumerate(syns):
        sName = s[0]+"-"+s[1]
        ax = axs.flatten()[i]
        mat = np.loadtxt(rPath+v+"/training/"+sName+"weights.vec")
        mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
        ws = mat[mask !=0]
        ws = ws[ws!=0]
        wM = ws.mean()
        wStd = ws.std()
        ws = ws[ws < wM+wStd*4]
        c, b, _ = ax.hist(ws, 50)
        ax.bar(x=wM,height=c.max(), width = b[1]*.6, color="red")

        form = tk.ScalarFormatter(useMathText=True)
        form.set_scientific(True)
        form.set_powerlimits((-1,1)) 

        ax.get_yaxis().set_major_formatter(form)
        ax.text(.5,.9,sName, transform=ax.transAxes)
    fig.suptitle(v, fontsize = 12)
    fig.tight_layout()
    fig.show()


In [ ]:
# Weights distributions - selected
lb = ["a)","b)","c)","d)","e)","f)","g)","h)"]
for v in vars:
    fig,axs = plt.subplots(4, 2, figsize = (9, 8))
    for i,sName in enumerate(["LGN-V1L4E", "LGN-V1L4I", "V1L4E-V1L4I","V1L4I-V1L4E", "V2L4E-V2L23E", "V1L4I-V1L4I", "V2L23E-V2L23I", "V2L23I-V2L23E"]):
        ax = axs.flatten()[i]
        mat = np.loadtxt(rPath+v+"/training/"+sName+"weights.vec")
        mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
        ws = mat[mask !=0]
        ws = ws[ws!=0]
        wM = ws.mean()
        wStd = ws.std()
        ws = ws[ws < wM+wStd*4]
        c, b, _ = ax.hist(ws, 50)
        ax.vlines(x=wM,ymin=0, ymax=c.max(), color="red")
        ax.text(-.2,1,lb[i], fontsize=16, transform=ax.transAxes)

        form = tk.ScalarFormatter(useMathText=True)
        form.set_scientific(True)
        form.set_powerlimits((-1,1)) 

        ax.get_yaxis().set_major_formatter(form)
        ax.text(.7,.9,sName, transform=ax.transAxes)
    fig.suptitle(v, fontsize = 12)
    fig.tight_layout()
    fig.show()


In [ ]:
# Weights stats
stats = []
for v in vars:
    means = []
    stds = []
    idx = []
    for i,s in enumerate(syns):
        sName = s[0]+"-"+s[1]
#        if s[2] != "FF": continue
        mat = np.loadtxt(rPath+v+"/training/"+sName+"weights.vec")
        mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
        ws = mat[mask !=0]
        means.append(ws.mean())
        stds.append(ws.std())
        idx.append(sName)
        #ws = ws[ws!=0]
    stats.append(pd.DataFrame({"mean":np.round(means,3), "std":np.round(stds,2)}, index=idx))
stats = pd.concat(stats, axis = 1, keys=vars)

In [ ]:
#inh = [ c for c in stats.index if c.split("-")[0][-1]=="I"]
#exc = [ c for c in stats.index if c.split("-")[0][-1]!="I"]

# Weights stats
stats = {v:{} for v in vars}
for v in vars:
    inh = []
    exc = []
    for i,s in enumerate(syns):
        sName = s[0]+"-"+s[1]
        mat = np.loadtxt(rPath+v+"/training/"+sName+"weights.vec")
        mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
        ws = mat[mask !=0]
        if s[0][-1] == "I": inh.append(ws.flatten())
        else: exc.append(ws)
    stats[v]["inh"] = np.concatenate(inh)
    stats[v]["exc"] = np.concatenate(exc)

In [ ]:
sp.stats.ttest_ind(stats["trace"]["inh"], stats["average"]["inh"], alternative = "less")

## Receptive Fields

In [ ]:
# Weights stats
stats = []
for v in vars:  
    means = []
    stds = []
    idx = []
    for i,s in enumerate(syns):
        sName = s[0]+"-"+s[1]
        if s[2] != "FF": continue
        mat = np.loadtxt(rPath+v+"/training/"+sName+"weights.vec")
        mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
        ws = mat[mask !=0]
        means.append(ws.mean())
        stds.append(ws.std())
        idx.append(sName)
        #ws = ws[ws!=0]
    stats.append(pd.DataFrame({"mean":np.round(means,3), "std":np.round(stds,2)}, index=idx))
stats = pd.concat(stats, axis = 1, keys=vars)
stats

In [ ]:
for v in vars[:1]:
    for s in syns[:2]:
        sName = s[0]+"-"+s[1]
        mat = np.loadtxt(rPath+v+"/training/"+sName+"weights.vec")
        mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
        fE, axE = plt.subplots(10, 10, figsize = (10,10))
        fE.suptitle(sName+" On", y=.92)    
        fI, axI = plt.subplots(10, 10, figsize = (10,10))
        fI.suptitle(sName+" Off", y=.92)
        fRF, axRF = plt.subplots(10, 10, figsize = (10,10))
        fRF.suptitle(sName+" Receptive Fields", y=.92)
        for i in range(100):
            rf = mat[:,100+i][mask[:,100+i]!=0.].reshape(10,10,2)
            exc = rf[:,:,0]/rf[:,:,0].max()
            inh = rf[:,:,1]/rf[:,:,1].max()
            axE[i//10][i%10].matshow(exc, cmap=plt.cm.Blues)
            axE[i//10][i%10].set_xticks([],[])
            axE[i//10][i%10].set_yticks([],[])
            axI[i//10][i%10].matshow(inh, cmap=plt.cm.Reds)
            axI[i//10][i%10].set_xticks([],[])
            axI[i//10][i%10].set_yticks([],[])
            axRF[i//10][i%10].matshow(inh-exc, cmap=plt.cm.bwr)
            axRF[i//10][i%10].set_xticks([],[])
            axRF[i//10][i%10].set_yticks([],[])

In [ ]:
lb = ["a)", "b)", "c)", "d)"]
lbi = 0
for v in vars:
    for s in syns[:2]:
        sName = s[0]+"-"+s[1]
        mat = np.loadtxt(rPath+v+"/training/"+sName+"weights.vec")
        mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
        fRF, axRF = plt.subplots(10, 10, figsize = (7,7))
        fRF.suptitle(s[1], y=.92)
        fRF.text(0.1,.9,lb[lbi], fontsize=16)
        lbi+=1
        for i in range(100):
            rf = mat[:,100+i][mask[:,100+i]!=0.].reshape(10,10,2)
            exc = rf[:,:,0]/rf[:,:,0].max()
            inh = rf[:,:,1]/rf[:,:,1].max()
            if True: # Separate versus joint scaling
                vrf = inh-exc
            else:
                vrf = rf[:,:,1]-rf[:,:,0]
                vrf /= np.abs(vrf.flatten()).max()

            axRF[i//10][i%10].matshow(vrf, cmap=plt.cm.bwr)
            axRF[i//10][i%10].set_xticks([],[])
            axRF[i//10][i%10].set_yticks([],[])

In [ ]:
from scipy.optimize import curve_fit as cf
import math as m
import random as r
from sklearn.metrics import mean_absolute_error as mae

In [ ]:

def gab(c, sx, sy, f, th, phi, A, B):
    x = c%10
    y = np.floor(c/10)
    x1 = x*np.cos(th)+y*np.sin(th)
    y1 = -x*np.sin(th)+y*np.cos(th)
    xp = -(x1**2/(2*sx**2))-(y1**2/(2*sy**2))
    return A*np.exp(xp)*np.cos(2*m.pi*f*x1-phi)+B

def find_ext(x, y):
    try:
        p,_,inf,_,_ = cf(gab,x,y, p0=[r.uniform(0,10),r.uniform(0,10), r.uniform(0,100), 6.28*r.random(), 6.28*r.random(), r.uniform(0,5), r.random()], full_output=True)
        err = mae(y, inf["fvec"])
        return p[0], p[1], p[3], err
    except RuntimeError:
        return 0,0,0,-1

for v in vars:
    for s in syns[:2]:
        sName = s[0]+"-"+s[1]
        mat = np.loadtxt(rPath+v+"/training/"+sName+"weights.vec")
        mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
        pars = {"sx":[], "sy":[], "th":[], "err":[]}
        for i in list(range(mat.shape[1])):
            rf = mat[:,i][mask[:,i]!=0.].reshape(10,10,2)
            exc = rf[:,:,0]/rf[:,:,0].max()
            inh = rf[:,:,1]/rf[:,:,1].max()
            rf = inh-exc
            x = [i for i in range(100)]
            y = rf.flatten()
            sx, sy, th = 0,0,0
            err = 1000000000
            for _ in range(100):
                nErr = -1
                while nErr == -1:
                    Tsx, Tsy, Tth, nErr = find_ext(x, y)
                if nErr < err:
                    err = nErr
                    sx = Tsx
                    sy = Tsy
                    th = Tth
            pars["sx"].append(sx)
            pars["sy"].append(sy)
            pars["th"].append(th)
            pars["err"].append(err)
        pd.DataFrame(pars).to_csv(rPath+v+"/training/"+sName+"extents.csv", index=False)

In [ ]:
fRF, axRF = plt.subplots(4, 2, figsize = (7,7))
for v in vars:
    for s in syns[:2][:-1]:
        sName = s[0]+"-"+s[1]
        tar = s[1]
        p = pd.read_csv(rPath+v+"/training/"+sName+"extents.csv")
        # Sign has no meaning
        p["sx"] = np.absolute(p["sx"].values)
        p["sy"] = np.absolute(p["sy"].values)
        # Extents 
        p.loc[p["sx"] > 10, "sx"] = np.nan
        p.loc[p["sy"] > 10, "sy"] = np.nan
p[["sx","sy"]]